# Imports 

In [2]:
import pandas as pd
pd.options.display.max_rows = 4
import json
import sqlite3 as sq3

# Data

In [3]:
with open(r"C:\Users\CPL17\OneDrive\Documents\Data\some_movies.json") as f:
    data = json.load(f)

In [4]:
df = pd.json_normalize(data, sep = "_")

df.head(1)

# Create Tables

In [10]:
movies = df[["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]].copy()

In [11]:
movies.release_date = pd.to_datetime(df.release_date)
movies.revenue = df.revenue/1000000
movies.budget = df.budget/1000000

In [12]:
votes = df[["id", "vote_count", "vote_average"]].copy()

In [13]:
genres = pd.json_normalize(data = data, record_path = "genres", meta = "id", record_prefix = "genre_")

In [14]:
prod = pd.json_normalize(data = data, record_path = "production_companies", meta = "id", record_prefix = "comp_")

# Connect to and add tables to a database

In [15]:
con = sq3.connect("movies.db")

In [16]:
movies.to_sql("Movies", con, index = False)
votes.to_sql("Votes", con, index = False)
genres.to_sql("Genres", con, index = False)
prod.to_sql("Prod", con, index = False)

In [ ]:
#Tables 

con.execute("Select * FROM sqlite_master").fetchall()

[('table',
  'Movies',
  'Movies',
  2,
  'CREATE TABLE "Movies" (\n"id" INTEGER,\n  "title" TEXT,\n  "revenue" REAL,\n  "budget" REAL,\n  "belongs_to_collection_name" TEXT,\n  "release_date" TIMESTAMP\n)'),
 ('table',
  'Votes',
  'Votes',
  3,
  'CREATE TABLE "Votes" (\n"id" INTEGER,\n  "vote_count" INTEGER,\n  "vote_average" REAL\n)'),
 ('table',
  'Genres',
  'Genres',
  4,
  'CREATE TABLE "Genres" (\n"genre_id" INTEGER,\n  "genre_name" TEXT,\n  "id" INTEGER\n)'),
 ('table',
  'Prod',
  'Prod',
  5,
  'CREATE TABLE "Prod" (\n"comp_id" INTEGER,\n  "comp_logo_path" TEXT,\n  "comp_name" TEXT,\n  "comp_origin_country" TEXT,\n  "id" INTEGER\n)')]

In [ ]:
#Table names 
con.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name").fetchall()

[('Genres',), ('Movies',), ('Prod',), ('Votes',)]

## Loading Data from SQLite Databases into DataFrames

In [ ]:
pd.read_sql("SELECT * FROM Movies", con).head()

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25 00:00:00
4,597,Titanic,1845.034188,200.0,None,1997-11-18 00:00:00


In [ ]:
df = pd.read_sql("SELECT * FROM Movies", con, index_col = "id", parse_dates = "release_date")
df.head(3)

,title,revenue,budget,belongs_to_collection_name,release_date
id,,,,,
299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24
19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10
140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15


##  Some Simple SQL Queries

In [ ]:
pd.read_sql("SELECT * \
            FROM Movies", con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10 00:00:00
...,...,...,...,...,...,...
16,321612,Beauty and the Beast,1263.521126,160.0,None,2017-03-16 00:00:00
17,260513,Incredibles 2,1241.891456,200.0,The Incredibles Collection,2018-06-14 00:00:00


In [ ]:
con.execute("SELECT sum(revenue) FROM Movies").fetchall()[0][0]

29748.575327000002

## Some more SQL Queries

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE revenue > 1500 AND budget < 200", con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,135397,Jurassic World,1671.713208,150.0,Jurassic Park Collection,2015-06-06 00:00:00
1,168259,Furious 7,1515.047671,190.0,The Fast and the Furious Collection,2015-04-01 00:00:00


In [ ]:
pd.read_sql("SELECT MIN(budget) FROM Movies WHERE revenue > 1250", con)

,MIN(budget)
0,125.0


## Join Queries

In [17]:
pd.read_sql("SELECT * \
            FROM Movies \
            JOIN Votes \
            ON Movies.id=Votes.id", con)

,id,title,revenue,budget,belongs_to_collection_name,release_date,id,vote_count,vote_average
0,299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24 00:00:00,299534,12607,8.3
1,19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10 00:00:00,19995,21000,7.4
...,...,...,...,...,...,...,...,...,...
16,321612,Beauty and the Beast,1263.521126,160.0,None,2017-03-16 00:00:00,321612,12203,6.9
17,260513,Incredibles 2,1241.891456,200.0,The Incredibles Collection,2018-06-14 00:00:00,260513,8147,7.5


In [18]:
pd.read_sql("SELECT Movies.id, Movies.title, Votes.vote_average \
            FROM Movies \
            JOIN Votes \
            ON Movies.id=Votes.id", con, index_col = "id")

,title,vote_average
id,,
299534,Avengers: Endgame,8.3
19995,Avatar,7.4
...,...,...
321612,Beauty and the Beast,6.9
260513,Incredibles 2,7.5


In [19]:
pd.read_sql("SELECT Movies.id, Movies.title, Movies.budget, Votes.vote_average \
            FROM Movies \
            JOIN Votes \
            ON Movies.id=Votes.id \
            WHERE Votes.vote_average > 8 \
            ORDER BY Movies.budget ASC", con, index_col = "id")

,title,budget,vote_average
id,,,
12445,Harry Potter and the Deathly Hallows: Part 2,125.0,8.1
299536,Avengers: Infinity War,300.0,8.3
299534,Avengers: Endgame,356.0,8.3


## Final Case Study

In [ ]:
# Highest revenue production companies

In [22]:
pd.read_sql("SELECT * FROM Prod", con)

,comp_id,comp_logo_path,comp_name,comp_origin_country,id
0,420,/hUzeosd33nzE5MCNsZxCGEKTXaQ.png,Marvel Studios,US,299534
1,444,/42UPdZl6B2cFXgNUASR8hSt9mpS.png,Dune Entertainment,US,19995
...,...,...,...,...,...
50,2,/wdrCwmRnLFJhEoH8GSfymY85KHT.png,Walt Disney Pictures,US,260513
51,3,/1TjvGVDMYsj6JBxOAkUHpPEwLf7.png,Pixar,US,260513


In [23]:
df = pd.read_sql("SELECT Prod.id, Prod.comp_name, Movies.revenue, Movies.title \
            FROM Prod \
            LEFT JOIN Movies \
            ON Prod.id=Movies.id", con)
df

,id,comp_name,revenue,title
0,299534,Marvel Studios,2797.800564,Avengers: Endgame
1,19995,Dune Entertainment,2787.965087,Avatar
...,...,...,...,...
50,260513,Walt Disney Pictures,1241.891456,Incredibles 2
51,260513,Pixar,1241.891456,Incredibles 2


In [26]:
df.groupby("comp_name").revenue.sum().sort_values(ascending = False)

comp_name
Walt Disney Pictures    9446.618940
Marvel Studios          9115.740912
                           ...     
Mandeville Films        1263.521126
Pixar                   1241.891456
Name: revenue, Length: 31, dtype: float64

In [ ]:
# Highest grossing genres

In [27]:
df2 = pd.read_sql("SELECT Genres.id, Genres.genre_name, Movies.revenue, Movies.title \
            FROM Genres \
            LEFT JOIN Movies \
            ON Genres.id=Movies.id", con)
df2

,id,genre_name,revenue,title
0,299534,Adventure,2797.800564,Avengers: Endgame
1,299534,Science Fiction,2797.800564,Avengers: Endgame
...,...,...,...,...
53,260513,Animation,1241.891456,Incredibles 2
54,260513,Family,1241.891456,Incredibles 2


In [28]:
df2.groupby("genre_name").revenue.sum().sort_values(ascending = False)

genre_name
Adventure    25124.972342
Action       21036.581432
                 ...     
Romance       3108.555314
Drama         1845.034188
Name: revenue, Length: 9, dtype: float64